<a href="https://colab.research.google.com/github/LuisOTSG/minicurso-cds/blob/main/aula04_minicurso_ds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Importação das bibliotecas, leitura dos arquivos e manipulação do conjunto de dados**

In [1]:
import pandas as pd
import io
import numpy as np
import plotly.express as px
import folium

from google.colab import files

In [2]:
arq = files.upload()

Saving AB_NYC_2019.csv to AB_NYC_2019.csv


In [3]:
df = pd.read_csv(io.BytesIO(arq['AB_NYC_2019.csv']))

In [4]:
df1 = df.dropna()

# **Respondendo mais perguntas de negócio**

## **Qual é o valor do aluguel (diária) mais caro de cada região da base de dados na cidade de Nova York?**

In [5]:
df1.groupby('neighbourhood_group')['price'].max().reset_index().sort_values(['price'], ascending=False)

,neighbourhood_group,price
1,Brooklyn,10000
3,Queens,10000
2,Manhattan,9999
0,Bronx,800
4,Staten Island,625


In [6]:
price_max_availability = df1.loc[df1['availability_365'] != 0].groupby('neighbourhood_group')['price'].max().reset_index()
price_max_availability.sort_values(['price'], ascending=False)

,neighbourhood_group,price
2,Manhattan,9999
1,Brooklyn,8000
3,Queens,2600
0,Bronx,800
4,Staten Island,625


In [7]:
px.bar(price_max_availability, x='neighbourhood_group', y='price',
       title= "Valor máximo diária por Bairro",
       labels= {'neighbourhood_group': "Bairro", 'price': "Valor máximo"})

## **Conseguimos saber onde estão localizados os imóveis com o valor do aluguel mais caro na cidade de Nova York?**

In [8]:
head = ['price', 'neighbourhood_group', 'latitude', 'longitude']
price_max_plot = df1.loc[:, head].groupby('neighbourhood_group').max().reset_index()
price_max_plot

,neighbourhood_group,price,latitude,longitude
0,Bronx,800,40.91306,-73.78158
1,Brooklyn,10000,40.73890,-73.85676
2,Manhattan,9999,40.87665,-73.90855
3,Queens,10000,40.79721,-73.71299
4,Staten Island,625,40.64779,-74.06356


In [9]:
map = folium.Map()

for index, location_info in price_max_plot.iterrows():
  folium.Marker([location_info['latitude'], location_info['longitude']],
                popup= location_info['neighbourhood_group']).add_to(map)

map

## **Conseguimos saber onde estão localizados os imóveis pelo seu tipo?**

In [10]:
head_type = ['neighbourhood_group', 'room_type', 'latitude', 'longitude']
type_location = df1.loc[:, head_type].sample(100)

type_location.loc[type_location['room_type'] == 'Private room', 'color'] = 'darkgreen'
type_location.loc[type_location['room_type'] == 'Entire home/apt', 'color'] = 'red'
type_location.loc[type_location['room_type'] == 'Shared room', 'color'] = 'blue'
type_location

,neighbourhood_group,room_type,latitude,longitude,color
44128,Brooklyn,Entire home/apt,40.71902,-73.96414,red
5910,Manhattan,Private room,40.72603,-73.98897,darkgreen
13711,Brooklyn,Entire home/apt,40.68507,-73.93888,red
32586,Brooklyn,Entire home/apt,40.66282,-73.95902,red
4873,Manhattan,Private room,40.71922,-73.98472,darkgreen
...,...,...,...,...,...
6388,Brooklyn,Entire home/apt,40.65500,-73.96109,red
10329,Manhattan,Private room,40.71846,-73.98672,darkgreen
31829,Brooklyn,Private room,40.64487,-73.90081,darkgreen
45036,Brooklyn,Entire home/apt,40.69402,-73.92378,red


In [11]:
map_type = folium.Map()

for index, location_info in type_location.iterrows():
  folium.Marker([location_info['latitude'], location_info['longitude']],
                popup= location_info['room_type'],
                icon= folium.Icon(color= location_info['color'])).add_to(map_type)

map_type